In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 
import datetime 

# Data Processing

## Loading data

In [2]:
df_hospital_org = pd.read_csv('/Users/jessie/Desktop/summer_project/Definitive_Healthcare%3A_USA_Hospital_Beds.csv', usecols = ['COUNTY_NAME', 'STATE_NAME', 'NUM_LICENSED_BEDS', 'NUM_STAFFED_BEDS', 'AVG_VENTILATOR_USAGE'])
df_social_mobility_org = pd.read_csv('/Users/jessie/Desktop/summer_project/social_mobility_data(up_to_date)/County.csv', usecols = ['CTNAME', 'STFIPS', 'date', 'Social distancing index', 'New cases/1000 people', 'Population density', '% people older than 60', 'COVID death rate'])
df_COVID19_org = pd.read_csv('/Users/jessie/Documents/GitHub/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
df_state_fips = pd.read_csv('/Users/jessie/Desktop/summer_project/social_mobility_data(up_to_date)/fips-states.csv', usecols = ['state_name', 'fips'])

In [3]:
print(df_hospital_org.shape)
df_hospital_org.head()

(6622, 5)


,COUNTY_NAME,STATE_NAME,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,AVG_VENTILATOR_USAGE
0,Maricopa,Arizona,62.0,62.0,0.0
1,Pima,Arizona,295.0,295.0,2.0
2,Fresno,California,54.0,54.0,2.0
3,New Haven,Connecticut,216.0,216.0,2.0
4,New Castle,Delaware,62.0,62.0,1.0


In [4]:
print(df_social_mobility_org.shape)
df_social_mobility_org.head()

(559276, 8)


,CTNAME,STFIPS,Social distancing index,date,% people older than 60,Population density,COVID death rate,New cases/1000 people
0,Autauga County,1,54,01/01/2020,19,91,0.0,0.0
1,Autauga County,1,25,01/02/2020,19,91,0.0,0.0
2,Autauga County,1,15,01/03/2020,19,91,0.0,0.0
3,Autauga County,1,33,01/04/2020,19,91,0.0,0.0
4,Autauga County,1,40,01/05/2020,19,91,0.0,0.0


In [5]:
print(df_COVID19_org.shape)
df_COVID19_org.head()

(3261, 177)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,247,247,247,253,257,267,280,280,280,280
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,30,31,31,31,31
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,6922,7066,7189,7250,7465,7537,7608,7683,7787,7916
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,81,81,81,81,81,90,92,98,111,111


In [6]:
print(df_state_fips.shape)
df_state_fips.head()

(51, 2)


,state_name,fips
0,Alabama,1
1,Alaska,2
2,Arizona,4
3,Arkansas,5
4,California,6


## Preprocess Hospital resources data

In [7]:
print(df_hospital_org.isnull().sum())
df_hospital = df_hospital_org.dropna()

COUNTY_NAME              28
STATE_NAME               28
NUM_LICENSED_BEDS       150
NUM_STAFFED_BEDS        238
AVG_VENTILATOR_USAGE     71
dtype: int64


In [8]:
print(df_hospital.dtypes)
print(df_hospital.describe())

df_hospital['COUNTY'] = df_hospital['COUNTY_NAME'] + str(',') + df_hospital["STATE_NAME"]
df_hospital_group = df_hospital.groupby(by = 'COUNTY').agg('sum').reset_index()

df_hospital_group.drop(df_hospital_group.loc[(df_hospital_group['NUM_STAFFED_BEDS']/ df_hospital_group['NUM_LICENSED_BEDS'] <= 0.1) 
                                             & (df_hospital_group['NUM_STAFFED_BEDS'] <= 10)].index, inplace = True)
print(len(df_hospital_group))
df_hospital_group.head()

COUNTY_NAME              object
STATE_NAME               object
NUM_LICENSED_BEDS       float64
NUM_STAFFED_BEDS        float64
AVG_VENTILATOR_USAGE    float64
dtype: object
       NUM_LICENSED_BEDS  NUM_STAFFED_BEDS  AVG_VENTILATOR_USAGE
count        6356.000000       6356.000000           6356.000000
mean          150.487256        128.924638              3.650409
std           183.131072        160.582287              5.517830
min             1.000000          1.000000              0.000000
25%            26.000000         25.000000              0.000000
50%            79.000000         66.000000              2.000000
75%           200.000000        168.000000              4.000000
max          2059.000000       2753.000000             61.000000
2491


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,COUNTY,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,AVG_VENTILATOR_USAGE
0,"Abbeville,South Carolina",25.0,25.0,2.0
1,"Acadia,Louisiana",170.0,144.0,3.0
2,"Accomack,Virginia",52.0,46.0,3.0
3,"Ada,Idaho",1311.0,1240.0,24.0
4,"Adair,Iowa",25.0,25.0,0.0


## Preprocess social mobility data

In [9]:
print(df_social_mobility_org.isnull().sum())
df_social_mobility = df_social_mobility_org.dropna()

CTNAME                     0
STFIPS                     0
Social distancing index    0
date                       0
% people older than 60     0
Population density         0
COVID death rate           0
New cases/1000 people      0
dtype: int64


In [10]:
df_social_mobility['CTNAME'] = df_social_mobility['CTNAME'].apply(lambda x: x.replace(str(' County'), ''))
df_social_mobility_merge = pd.merge(df_social_mobility, df_state_fips, left_on = 'STFIPS', right_on = 'fips')
df_social_mobility_merge['COUNTY'] = df_social_mobility_merge['CTNAME'] + str(',') + df_social_mobility_merge['state_name']
df_social_mobility_merge.drop(columns = ['CTNAME', 'STFIPS', 'state_name', 'fips'], inplace = True)
df_social_mobility_merge.head()
#df_social_mobility_group = df_social_mobility_merge.groupby(by = 'date')

#print(df_social_mobility_group.size())
#df_social_mobility_current = df_social_mobility_group.get_group('06/22/2020')

# print(df_social_mobility_current.dtypes)
# print(df_social_mobility_current.describe())
# df_social_mobility_current.head()
#df_social_mobility_current['Population density'].plot.hist()

#df_social_mobility_current.loc[df_social_mobility_current['Population density'] == 0]
#len(df_social_mobility_group.get_group('01/01/2020')['COUNTY'].unique())

,Social distancing index,date,% people older than 60,Population density,COVID death rate,New cases/1000 people,COUNTY
0,54,01/01/2020,19,91,0.0,0.0,"Autauga,Alabama"
1,25,01/02/2020,19,91,0.0,0.0,"Autauga,Alabama"
2,15,01/03/2020,19,91,0.0,0.0,"Autauga,Alabama"
3,33,01/04/2020,19,91,0.0,0.0,"Autauga,Alabama"
4,40,01/05/2020,19,91,0.0,0.0,"Autauga,Alabama"


## Preprocess COVID19 data

In [13]:
print(df_COVID19_org.isnull().sum())
df_COVID19 = df_COVID19_org.dropna()
print(df_COVID19.shape)

UID                0
iso2               0
iso3               0
code3              0
FIPS              10
Admin2             7
Province_State     0
Country_Region     0
Lat                0
Long_              0
Combined_Key       0
1/22/20            0
1/23/20            0
1/24/20            0
1/25/20            0
1/26/20            0
1/27/20            0
1/28/20            0
1/29/20            0
1/30/20            0
1/31/20            0
2/1/20             0
2/2/20             0
2/3/20             0
2/4/20             0
2/5/20             0
2/6/20             0
2/7/20             0
2/8/20             0
2/9/20             0
                  ..
6/6/20             0
6/7/20             0
6/8/20             0
6/9/20             0
6/10/20            0
6/11/20            0
6/12/20            0
6/13/20            0
6/14/20            0
6/15/20            0
6/16/20            0
6/17/20            0
6/18/20            0
6/19/20            0
6/20/20            0
6/21/20            0
6/22/20      

In [14]:
df_COVID19.columns.to_series()

UID                          UID
iso2                        iso2
iso3                        iso3
code3                      code3
FIPS                        FIPS
Admin2                    Admin2
Province_State    Province_State
Country_Region    Country_Region
Lat                          Lat
Long_                      Long_
Combined_Key        Combined_Key
1/22/20                  1/22/20
1/23/20                  1/23/20
1/24/20                  1/24/20
1/25/20                  1/25/20
1/26/20                  1/26/20
1/27/20                  1/27/20
1/28/20                  1/28/20
1/29/20                  1/29/20
1/30/20                  1/30/20
1/31/20                  1/31/20
2/1/20                    2/1/20
2/2/20                    2/2/20
2/3/20                    2/3/20
2/4/20                    2/4/20
2/5/20                    2/5/20
2/6/20                    2/6/20
2/7/20                    2/7/20
2/8/20                    2/8/20
2/9/20                    2/9/20
          

In [16]:
df_COVID19_demo = df_COVID19[['Admin2', 'Province_State', 'Lat', 'Long_']]
df_COVID19_demo['COUNTY'] = df_COVID19_demo['Admin2'] + str(',') + df_COVID19_demo['Province_State']
df_COVID19_demo.drop(columns = ['Admin2', 'Province_State'], inplace = True)
df_COVID19_demo.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Lat,Long_,COUNTY
5,32.539527,-86.644082,"Autauga,Alabama"
6,30.727750,-87.722071,"Baldwin,Alabama"
7,31.868263,-85.387129,"Barbour,Alabama"
8,32.996421,-87.125115,"Bibb,Alabama"
9,33.982109,-86.567906,"Blount,Alabama"


In [18]:
start = df_COVID19.columns.get_loc('3/1/20')
end = df_COVID19.columns.get_loc('6/26/20')

df_COVID19_daily = df_COVID19.iloc[:, start: end + 1]
df_COVID19_daily.head()

,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,...,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20
5,0,0,0,0,0,0,0,0,0,0,...,369,394,405,425,428,436,447,463,473,482
6,0,0,0,0,0,0,0,0,0,0,...,377,386,398,405,415,422,435,449,462,500
7,0,0,0,0,0,0,0,0,0,0,...,250,262,265,271,271,276,279,287,303,309
8,0,0,0,0,0,0,0,0,0,0,...,115,118,123,123,124,126,132,138,146,150
9,0,0,0,0,0,0,0,0,0,0,...,121,127,136,140,146,150,156,165,173,181


In [19]:
df_COVID19 = pd.concat([df_COVID19_demo, df_COVID19_daily], axis = 1)
df_COVID19.head()

,Lat,Long_,COUNTY,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,...,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20
5,32.539527,-86.644082,"Autauga,Alabama",0,0,0,0,0,0,0,...,369,394,405,425,428,436,447,463,473,482
6,30.727750,-87.722071,"Baldwin,Alabama",0,0,0,0,0,0,0,...,377,386,398,405,415,422,435,449,462,500
7,31.868263,-85.387129,"Barbour,Alabama",0,0,0,0,0,0,0,...,250,262,265,271,271,276,279,287,303,309
8,32.996421,-87.125115,"Bibb,Alabama",0,0,0,0,0,0,0,...,115,118,123,123,124,126,132,138,146,150
9,33.982109,-86.567906,"Blount,Alabama",0,0,0,0,0,0,0,...,121,127,136,140,146,150,156,165,173,181


## Get 5 nearby county

In [21]:
df_COVID19.head()

,Lat,Long_,COUNTY,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,...,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20
5,32.539527,-86.644082,"Autauga,Alabama",0,0,0,0,0,0,0,...,369,394,405,425,428,436,447,463,473,482
6,30.727750,-87.722071,"Baldwin,Alabama",0,0,0,0,0,0,0,...,377,386,398,405,415,422,435,449,462,500
7,31.868263,-85.387129,"Barbour,Alabama",0,0,0,0,0,0,0,...,250,262,265,271,271,276,279,287,303,309
8,32.996421,-87.125115,"Bibb,Alabama",0,0,0,0,0,0,0,...,115,118,123,123,124,126,132,138,146,150
9,33.982109,-86.567906,"Blount,Alabama",0,0,0,0,0,0,0,...,121,127,136,140,146,150,156,165,173,181


In [39]:
df_county = df_COVID19[['COUNTY', 'Lat', 'Long_']]

In [210]:
def get_NearbyCounty(df, row):
    df['Distance'] = np.sqrt((df['Lat'] - row['Lat'])**2 + (df['Long_'] - row['Long_'])**2)
    df.sort_values(by = ['Distance'], inplace = True)
    county_array = np.array(df['COUNTY'].iloc[1:3].get_values())
    return [county_array]

In [212]:
#df_sub = df_county.iloc[0:5]
#df_sub.loc[df_sub['COUNTY'] == 'Baldwin,Alabama']
df_county['NEARBY_COUNTY'] = df_county.apply(lambda x: get_NearbyCounty(df_county.copy(), x), axis = 1)
df_county.head()
#df_sub.apply(lambda x: return(x['COUNTY']), axis = 1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,COUNTY,Lat,Long_,NEARBY_COUNTY
5,"Autauga,Alabama",32.539527,-86.644082,"[[Chilton,Alabama, Lowndes,Alabama]]"
6,"Baldwin,Alabama",30.727750,-87.722071,"[[Escambia,Florida, Mobile,Alabama]]"
7,"Barbour,Alabama",31.868263,-85.387129,"[[Quitman,Georgia, Henry,Alabama]]"
8,"Bibb,Alabama",32.996421,-87.125115,"[[Perry,Alabama, Chilton,Alabama]]"
9,"Blount,Alabama",33.982109,-86.567906,"[[Cullman,Alabama, St. Clair,Alabama]]"


In [213]:
df_county.shape

(3244, 4)

In [214]:
df_county.drop(columns = ['Lat', 'Long_'], inplace = True)
df_county.head()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,COUNTY,NEARBY_COUNTY
5,"Autauga,Alabama","[[Chilton,Alabama, Lowndes,Alabama]]"
6,"Baldwin,Alabama","[[Escambia,Florida, Mobile,Alabama]]"
7,"Barbour,Alabama","[[Quitman,Georgia, Henry,Alabama]]"
8,"Bibb,Alabama","[[Perry,Alabama, Chilton,Alabama]]"
9,"Blount,Alabama","[[Cullman,Alabama, St. Clair,Alabama]]"


In [217]:
dict_NearbyCounty = df_county.set_index('COUNTY')['NEARBY_COUNTY'].to_dict()
len(dict_NearbyCounty['Autauga,Alabama'][0])

2

In [209]:
df_sub.iloc[0]['COUNTY'] in df_sub["NEARBY_COUNTY"].iloc[3][0]

True

## Altering structure for COVID19 dataset

In [249]:
date_array = np.array(df_COVID19_daily.columns)

In [250]:
#datetime_array = [datetime.datetime.strptime(date + '20',"%m/%d/%Y") for date in date_array]

In [274]:
df_COVID19_tempo = df_COVID19.drop(columns = ['Lat', 'Long_'])
df_COVID19_tempo.set_index('COUNTY', inplace = True)

In [275]:
df_COVID19_tempo.head()

,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,...,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20
COUNTY,,,,,,,,,,,,,,,,,,,,,
"Autauga,Alabama",0,0,0,0,0,0,0,0,0,0,...,369,394,405,425,428,436,447,463,473,482
"Baldwin,Alabama",0,0,0,0,0,0,0,0,0,0,...,377,386,398,405,415,422,435,449,462,500
"Barbour,Alabama",0,0,0,0,0,0,0,0,0,0,...,250,262,265,271,271,276,279,287,303,309
"Bibb,Alabama",0,0,0,0,0,0,0,0,0,0,...,115,118,123,123,124,126,132,138,146,150
"Blount,Alabama",0,0,0,0,0,0,0,0,0,0,...,121,127,136,140,146,150,156,165,173,181


In [334]:
stack_array = df_COVID19_tempo.stack().values.tolist()
len(stack_array)

382792

In [293]:
county_array = np.array([])

In [294]:
#county_array = np.append(county_array, [np.repeat(county, len(date_array)) for county in df_COVID19_tempo.index])

In [329]:
county_array = np.repeat(df_COVID19_tempo.index, len(date_array)).tolist()

In [330]:
len(county_array)

382792

In [319]:
cum_date_array = date_array.tolist()
cum_date_array = cum_date_array * len(df_COVID19_tempo.index)

In [320]:
len(cum_date_array)

382792

In [332]:
stack_array

array([0, 0, 0, ..., 0, 0, 0])

In [336]:
df_COVID19_final = pd.DataFrame({'COUNTY': county_array, 'DATE': cum_date_array, 'DAILY_INCREASE': stack_array})

In [340]:
df_COVID19_final[(df_COVID19_final['COUNTY'] == 'Autauga,Alabama') & (df_COVID19_final['DATE'] == '6/17/20')]

,COUNTY,DATE,DAILY_INCREASE
108,"Autauga,Alabama",6/17/20,369


## Combining datasets

In [ ]:
print(len(df_hospital_group['COUNTY'].unique()))
print(len(df_social_mobility_current['COUNTY'].unique()))
print(len(df_COVID19['COUNTY'].unique()))

In [ ]:
print(len(df_hospital_group))
print(len(df_social_mobility_current))
print(len(df_COVID19))

In [ ]:
df_basic = pd.merge(df_COVID19, pd.merge(df_hospital_group, df_social_mobility_current, on = ['COUNTY']), on = ['COUNTY'])
#print(len(df_basic))
print(df_basic.iloc[0])
df_clean = df_basic.drop(columns = ['Admin2', 'Province_State', 'CTNAME', 'STFIPS', 'date', 'state_name', 'fips'])
state_col = df_clean.pop('COUNTY')
df_clean.insert(0, 'COUNTY', state_col)
df_clean.to_csv('./social_mobility_and _covid19.csv')
print(df_clean.shape)
df_clean.head()

In [ ]:
county_list = df_hospital["COUNTY_NAME"].unique()
np.any([type(i) == float for i in county_list])
check = [type(i) == float for i in county_list]
for i in range(len(check)):
    if check[i] == True:
        print(i)
        print(county_list[i])
print(county_list[1410:1420])
#len(county_list)

In [ ]:
df_social_mobility = pd.read_csv('/Users/jessie/Desktop/summer_project/social_mobility_data(up_to_date)/County.csv')

In [ ]:
df_social_mobility.head()

In [ ]:
df_social_mobility.iloc[0]

In [ ]:
df_social_mobility.isnull().sum()

In [ ]:
len(df_social_mobility['CTNAME'].unique())
type(df_social_mobility['CTNAME'].unique())
print(df_social_mobility['CTNAME'].unique())

In [ ]:
np.intersect1d(df_hospital['COUNTY_NAME'].unique(), df_social_mobility['CTNAME'].unique())

In [ ]:
pwd

In [ ]:
cd Desktop/summer_project/

In [ ]:
cd '/Users/jessie/Documents/GitHub/'

In [ ]:
ls

In [ ]:
cd COVID-19

In [ ]:
ls

In [ ]:
cd csse_covid_19_data/

In [ ]:
ls

In [ ]:
cd  csse_covid_19_time_series

In [ ]:
ls